In [ ]:
%matplotlib notebook
import mne
import os.path as op

epochs_clean = mne.read_epochs('../data/derived/cleaned_epo.fif')
#epochs_clean.plot_sensors()



In [ ]:
subject='nathalie'
subjects_dir='../data/raw/MRI'
derived='../data/derived/'
trans = '../data/derived/nathalie-trans.fif'
src = mne.read_source_spaces(derived + 'nathalie-oct6-src.fif')

In [ ]:
mne watershed_bem -s nathalie
model = mne.make_bem_model('nathalie', subjects_dir='../data/raw/MRI/')
mne.write_bem_surfaces('../data/derived/nathalie-bem.fif', model) 
bem_sol = mne.make_bem_solution(model)  
mne.write_bem_solution('../data/derived/nathalie-bem-sol.fif', bem_sol) 

In [ ]:
%matplotlib notebook

mne.viz.plot_bem(subject=subject, subjects_dir=subjects_dir,
                 brain_surfaces='white', orientation='coronal')

# Aligning coordinate frames

In [ ]:
 #mne.gui.coregistration(subject='nathalie', subjects_dir=subjects_dir)
from mne.viz import plot_alignment
from mayavi import mlab
mne.viz.plot_alignment(epochs_clean.info, trans=trans, subject='nathalie', src=src,
                       subjects_dir=subjects_dir,  dig=False,
                     eeg=['original', 'projected'], meg=[],
                        coord_frame='head')

In [ ]:
import numpy as np  # noqa
from mayavi import mlab  # noqa
from surfer import Brain  # noqa

brain = Brain(subject, 'lh', 'inflated', subjects_dir=subjects_dir)
surf = brain.geo['lh']

vertidx = np.where(src[0]['inuse'])[0]

mlab.points3d(surf.x[vertidx], surf.y[vertidx],
              surf.z[vertidx], color=(1, 1, 0), scale_factor=1.5)

In [ ]:
#conductivity = (0.3,)  # for single layer
conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject= subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [ ]:
fwd = mne.make_forward_solution(epochs_clean.info, trans=trans, src=src, bem=bem,
                                meg=False, eeg=True, mindist=5.0, n_jobs=2)
print(fwd)

In [ ]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)
leadfield = fwd_fixed['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

In [ ]:
 mne.write_forward_solution('../data/derived/nathalie-fwd.fif',fwd_fixed)


In [ ]:
mne.read_forward_solution('../data/derived/nathalie-fwd.fif')
